In [ ]:
# ПОРІВНЯННЯ АЛГОРИТМІВ
from tkinter import *
import tkinter as tk
from PIL import ImageTk, Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib

import math
from math import e, pi, sin, cos, log

from scipy.fftpack import fft, ifft
import time
import sys
%matplotlib inline
matplotlib.use('TkAgg')

def W(N, l):
    m = (-pi*l/N)*2j
    return e ** (m)

def dpf(x):
    N=len(x)
    S=[]
    for i in range(N):
        Sn=0
        for k in range(N):
            Sn=Sn + x[k]*W(N, k*i)
        S.append(Sn)
    return S

#обернене ДПФ
def ndpf(x):
    N=len(x)
    S=[]
    for i in range(N):
        Sn=0
        for k in range(N):
            Sn=Sn + x[k]*W(N, -k*i)
        S.append(Sn/N)      
    return S

def dpf1(ff):
    N = len(ff)
    if N <= 1:
        return ff
    fp = dpf1(ff[0::2])
    fnp = dpf1(ff[1::2])
    NP = [W(N, k) * fnp[k] for k in range(N // 2)]
    Cn = [fp[k] + NP[k] for k in range(N // 2)] + [fp[k] - NP[k] for k in range(N // 2)]
    return Cn

#степінь 2?
def step(a):
    p = log(a, 2)
    if p == int(p):
        return True
    else:
        return False
    
#модуль комплексного числа, із врахуванням знаку
def Real(x):
    fff=[]
    for i in range(Z):
        if x[i].real>0:
            fff.append(abs(x[i]))
        else:
            fff.append(-abs(x[i]))
    return fff

#пилкоподібна функція фази, - для обчислення початкової фази вхідної функції відповідної гарманіки
#фаза лежить в межах від -π до π
def faza(d):
    if d>0:
        while d>pi:
            d=d-2*pi
    elif d<0:
        while d<-pi:
            d=d+2*pi
    return d

def calc():
    global Z, Z2, t_1, t_2, t_3, x1, x2, x3, y1, y2, y3, fps, v, t, f
    global fk0, fk, t0, f1
    
    text1.delete('1.0',tk.END)
    text1.insert('1.0','Виведення:\n')
    
    #v - частота основного тону в Гц (власна частота);
    try:
        v = float(entry1.get())
    except ValueError:
        v=4
        entry1.delete(0,tk.END)
        entry1.insert(0,v)

    # fps -- частота дискретизації сигналу - кількість значень сигналу у одній секунді
    # frames per second - кадрів в секунду 
    try:
        fps = float(entry2.get())
    except ValueError:
        fps=256
        entry2.delete(0,tk.END)
        entry2.insert(0,fps)

    # fps>=2*v -- теорема Найквіста-Котельникова
    # щоб повністю відтворити сигнал необхідно, щоб 
    # частота дискретизації (згідно теореми) була принаймні в 2 рази більшою 
    # за максимальну частоту звуку, який людина може сприймати (20-20000Гц). Тобто fps>=20000*2=40000
    # 32768=2^15

    # період вибірки - відстань в часі між 2-ма найблищими значеннями
    dT=1/fps

    # тривалість сигналу у секундах
    try:
        T = float(entry3.get())
    except ValueError:
        T=0.5
        entry3.delete(0,tk.END)
        entry3.insert(0,T)

    T0 = T
    # має виконуватися умова: загальна кількість точок у сигналі Z=fps*T=(2^p), отже для тривалості сигналу T=(2^p)/fps

    #Z - загальна кількість точок у сигналі; кількість значень на відрізку v*T
    Z=int(fps*T)
    u='Z=%s  2^p--%s'%(Z, step(Z))
    print('Z=',Z, '2^p--',step(Z))
    text1.insert('2.0',u+'\n')

    #перевірка умови для Z (скорочення тривалості сигналу)
    if not step(Z):
        p=int(log(Z, 2))
        T=(2**p)/fps
        u='T0= %s  T= %s' %(T0, T)
        text1.insert('3.0',u+'\n')
        print('T0=',T0, 'T=',T)
      
        Z=int(fps*T)
        u='Z=%s  2^p--%s'%(Z, step(Z))
        print('Z=',Z, '2^p--',step(Z))
        text1.insert('4.0',u+'\n')

    # кількість періодів функції f (бажано, щоб була цілим числом, інакше - розтікання спектру!)
    print('кількість періодів функції f--',v*T)
    u='кількість періодів функції f-- %s'% (v*T)
    text1.insert('5.0',u+'\n')

    # вісь часу
    # ділимо відрізок 0..T на Z рівних частин
    # [0..Z]*dT
    t=np.arange(Z)*dT
    #t = np.linspace(0, T, Z) - (однаковий крок) не працює (дає багато 'шуму' після аналізу!?)

    f=[]
    # досліджувана функція
    for i in range(Z):   
        f.append(2*sin(2*pi*v*t[i]+8)+3*cos(4*pi*v*t[i]-100)+1.5*sin(6*pi*v*t[i]+100)-7*cos(8*pi*v*t[i]))
        #f(t)=2sin(2π(1ν)t+8)+3cos(2π(2ν)t-100)+1.5sin(2π(3ν)t+100)-7cos(2π(4ν)t)
    #f=2*np.sin(2*pi*v*t+8)+3*np.cos(4*pi*v*t-100)+1.5*np.sin(6*pi*v*t+100)-7*np.cos(8*pi*v*t) - без циклу for...
    
    u='т. Н.-К. %s >= %s -- %s' %(fps, 2*(4*v), fps>=2*(4*v))
    text1.insert('6.0',u+'\n')
    print('т. Н.-К.',fps, '>=', 2*(4*v), fps>=2*(4*v))
  
    u='v= %s Гц;   T= %s с;   Z= %s' % (v,T,Z)
    text1.insert('7.0',u+'\n')
    #u='v='+ str(v)+'Гц;   T='+str(T)+'с;   Z='+str(Z)
    print(u)
    label4['text']=u

    #поточна глибина рекурсії
    # print(sys.getrecursionlimit())
    # sys.setrecursion(5000)
    # sys.setrecursionlimit(10000)
    
    start = time.time()
    y_1=dpf(f)
    end = time.time()
    t_1=end - start
    u="--- %s seconds dpf---" % t_1
    print("--- %s seconds dpf---" % t_1)
    text1.insert('8.0',u+'\n')

    start = time.time()
    y_2=dpf1(f)
    end = time.time()
    t_2=end - start
    u="--- %s seconds dpf1---" % t_2
    print("--- %s seconds dpf1---" % t_2)
    text1.insert('9.0',u+'\n')

    start = time.time()
    y_3=fft(f)
    end = time.time()
    t_3=end - start
    u="--- %s seconds fft---" % t_3
    print("--- %s seconds fft---" % t_3)
    text1.insert('10.0',u+'\n')

    #обернене ДПФ
    start = time.time()
    y_4=ndpf(y_3)
    end = time.time()
    t_4=end - start
    u="--- %s seconds ndpf ---" % t_4
    print("--- %s seconds ndpf ---" % t_4)
    text1.insert('11.0',u+'\n')

    #вбудовананий модуль оберненого ДПФ
    start = time.time()
    y_5=ifft(y_3)
    end = time.time()
    t_5=end - start
    u="--- %s seconds ifft ---" % t_5
    print("--- %s seconds ifft ---" % t_5)
    text1.insert('12.0',u+'\n')

    L1=len(y_1)
    L2=len(y_2)
    L3=len(y_3)

    #y[i] - амплітуда і-ї частоти ("мода")
    y1=[]
    y2=[]
    y3=[]

    for i in range(L1):
        y1.append(abs(y_1[i])/(L1/2))
    for i in range(L2):
        y2.append(abs(y_2[i])/(L2/2))

    #для фази вихідного сигналу 
    fk=[]
    #максимальне значення аргумента комплексного числа
    maxY_3=max(map(abs, y_3/(L3/2)))
    #s=list(map(abs, y_3))
    #print(s)
    #print('max', maxY_3)
    for i in range(L3):
        r=y_3[i]/(L3/2)
        y3.append(abs(r))
        #занулюємо дуже малі значення, інакше буде зашумленмй спектр фаз
        if abs(r)<maxY_3/10000:
            fk.append(0)
        else:
            #фаза у градусах
            fk.append(math.atan2(r.imag, r.real)*180/pi)

    #обернене ДПФ
    f1=Real(y_5)
        
    #осі частот, відповідно до кількості вхідних значень; Vi=i/T
    x1 = np.linspace(0, int((L1)/T), L1)
    x2 = np.linspace(0, int((L2)/T), L2)
    x3 = np.linspace(0, int((L3)/T), L3)

    #f(t)=2sin(2π(1ν)t+8)+3cos(2π(2ν)t-100)+1.5sin(2π(3ν)t+100)-7cos(2π(4ν)t)
    #початкові фази і-ї гармоніки вхідного сигналу у градусах; f=Acos(2πνt+φ)+... (ряд у косинусах); sinx=cos(x-π/2)
    fk0=[0 for i in range(Z)]

    #крок частоти 
    dv=(L1)/(T*Z)
    #кількість кроків частот, які вкладаються в основному тоні (може вийти не ціла кількість кроків - дробова частота)
    vv=round(v/dv)
    u='кількість кроків до основного тону %s; точно %s' %(vv, v/dv)
    print('кількість кроків до основного тону', vv, '; точно', v/dv)
    text1.insert('13.0',u+'\n')

    fk0[1*vv]=faza(8-pi/2)*180/pi
    fk0[2*vv]=faza(-100)*180/pi
    fk0[3*vv]=faza(100-pi/2)*180/pi
    fk0[4*vv]=pi*180/pi

    #fk0[1*v]=math.acos(cos(8-pi/2))
    #fk0[2*v]=math.acos(cos(-100))
    #fk0[3*v]=math.acos(cos(100-pi/2))
    #fk0[4*v]=math.acos(-cos(0))
    
    #в зв'язку із симетрією ДПФ
    Z2=int(Z/2)
    
    button1['state'] = tk.NORMAL
    button2['state'] = tk.NORMAL
    button3['state'] = tk.NORMAL
    button4['state'] = tk.NORMAL
    button5['state'] = tk.NORMAL
    button6['state'] = tk.NORMAL
    
    #один період функції (кількість точок в одному періоді) t0=tmax=1/min(v,2v,3v,4v)
    t0=int(fps/v)
    

def graph():
    f, a = plt.subplots(1, 1)
    house_prices = np.random.normal(0., 1, 1000)
    a.hist(house_prices)
    plt.show()

def graph1():
    # figure - вікно, рисунок на якому будуть графіки
    # axes - сам графік, система координат
    fig = plt.figure()
    # розкреслюємо рисунок на 3 частини
    # вставляємо графік в першу комірку
    ax1 = fig.add_subplot(3, 1, 1)
    # вставляємо графік в другу комірку
    ax2 = fig.add_subplot(3, 1, 2)
    # вставляємо графік в третю комірку
    ax3 = fig.add_subplot(3, 1, 3)

    #r-(raw) необроблені рядки
    #print(r'123\n567')

    ax1.set_title(r'АЧХ_dpf_'+str(t_1)+'s', fontsize=10, x=0.5, y=0)
    # Встановлюємо інтервал основних поділок
    ax1.yaxis.set_major_locator(ticker.MultipleLocator(1))
    # Встановлюємо інтервал допоміжних поділок
    ax1.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))
    ax1.bar(x1[:Z2], y1[:Z2])

    ax2.set_title(r'АЧХ_dpf1_'+str(t_2)+'s', fontsize=10, x=0.5, y=0)
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax2.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))
    ax2.bar(x2[:Z2], y2[:Z2])

    ax3.set_title(r'АЧХ_fft_'+str(t_3)+'s', fontsize=10, x=0.5, y=0)
    ax3.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax3.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))
    ax3.bar(x3[:Z2], y3[:Z2])
    fig.savefig('saved_figure_3_1.png', dpi = 1000)
    
def graph2():
    #логарифмічна* шкала осей координат (на графіку буде видно "шуми" ДПФ)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(r'АЧХ з прологарифмованими шкалами осей координат', fontsize=10, x=0.5, y=0)
    ax.plot(x3[:Z2], y3[:Z2], color='blue',  linestyle='--')
    ax.set_yscale('log')
    ax.set_xscale('log')
    fig.savefig('saved_figure_3_2.png', dpi = 1000)
    
def graph3():
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(r'Графік вх. функції f', fontsize=10, x=0.5, y=0)
    
    
    ax.plot(t[:t0], f[:t0], color='blue',  marker='o', linestyle='--')
    ax.set_xlabel('час, с')
    fig.savefig('saved_figure_3_3.png', dpi = 1000)
    
def graph4():
    #Порівняння графіка вх. функції із графіком оберненого ДПФ
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(r'Час', fontsize=10, x=0.5, y=0)
    ax.plot(t[:t0], f[:t0], color='blue', marker='o', label="Графік функції f", linewidth=4)
    plt.legend()
    ax.plot(t[:t0], f1[:t0], color='yellow', marker='+', linestyle='-',label="Обернене ДПФ")
    plt.legend()
    fig.savefig('saved_figure_3_4.png', dpi = 1000)
    
def graph5():
    #спектр фаз коливаньм вх. функції 
    fig = plt.figure()
    ax1 = fig.add_subplot(2, 1, 1)
    ax2 = fig.add_subplot(2, 1, 2)

    ax1.set_title(r'Спектр фаз вх. функції $\phi_{вх.}(\nu)$', fontsize=10, x=0.5, y=0)
    ax1.yaxis.set_major_locator(ticker.MultipleLocator(45))
    ax1.yaxis.set_minor_locator(ticker.MultipleLocator(15))
    ax1.bar(x3[:Z2], fk0[:Z2])

    #спектр фаз коливаньм вих. функції 
    ax2.set_title(r'Спектр фаз вих. функції $\phi_{вих.}(\nu)$', fontsize=10, x=0.5, y=0)
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(45))
    ax2.yaxis.set_minor_locator(ticker.MultipleLocator(15))
    ax2.bar(x3[:Z2], fk[:Z2])
    
    fig.savefig('saved_figure_3_5.png', dpi = 1000)
    
def graph6():
    #Фазо-частотна характеристика - зсув фази вихідного сигналу по відношенню до вхідного сигналу як функція частоти
    fk1=[]
    for i in range(Z2):
        fk1.append(round(fk[i]-fk0[i],5))
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(r'ФЧХ', fontsize=10, x=0.5, y=0)
    ax.plot(x3[:Z2], fk1[:Z2], color='yellow', marker='+', linestyle='-',label=r'$\Delta \phi = \phi_{вих.}-\phi_{вх.}$' )
    plt.legend()
    fig.savefig('saved_figure_3_6.png', dpi = 1000)
    
    
#Створюємо головне вікно
win=tk.Tk()
win.title('Порівняння')
win.geometry("600x500")

label0=tk.Label(win, text='---------------------')
label0.grid(row=0, column=0, columnspan=3)

label1=tk.Label(win, text='v, Гц')
label1.grid(row=1, column=0)
label2=tk.Label(win, text='fps, точ./c')
label2.grid(row=1, column=1)
label3=tk.Label(win, text='T, c')
label3.grid(row=1, column=2)

#створюємо віджет Entry, - об'єкт (змінна) entry1

#v=
entry1=tk.Entry(win)
entry1.grid(row=2, column=0)
entry1.delete(0,tk.END)
entry1.insert(0,4)

#fps=
entry2=tk.Entry(win)
entry2.grid(row=2, column=1)
entry2.delete(0,tk.END)
entry2.insert(0,256)

#T=
entry3=tk.Entry(win)
entry3.grid(row=2, column=2)
entry3.delete(0,tk.END)
entry3.insert(0,0.5)
#викликаємо метод, який дозволяє розмістити об'єкт на вікні 
#у відповідних рядку та стовпці головного вікна відносно лівого верхнього кута

label4=tk.Label(win, text='---')
label4.grid(row=3, column=0, columnspan=3)

button0=tk.Button(win, text='Обчислити',
                  command=calc)
button0.grid(row=4, column=0, columnspan=3)

button1=tk.Button(win, text='Графіки АЧХ', state=tk.DISABLED,
                  command=graph1)
button1.grid(row=5, column=0)

button2=tk.Button(win, text='Графік АЧХ_LOG', state=tk.DISABLED,
                  command=graph2)
button2.grid(row=5, column=1)

button3=tk.Button(win, text='Графік f', state=tk.DISABLED,
                  command=graph3)
button3.grid(row=5, column=2)

button4=tk.Button(win, text='Графік f і об. ДПФ', state=tk.DISABLED,
                  command=graph4)
button4.grid(row=6, column=0)

button5=tk.Button(win, text='Графіки спектрів фаз', state=tk.DISABLED,
                  command=graph5)
button5.grid(row=6, column=1)

button6=tk.Button(win, text='Графік ФЧХ', state=tk.DISABLED,
                  command=graph6)
button6.grid(row=6, column=2)

text1 = tk.Text(win, height=14, width=50, font='14')
text1.grid(row=7, column=0, columnspan=3)


win.grid_columnconfigure(0, weight=1)
win.grid_columnconfigure(1, weight=1)
win.grid_columnconfigure(2, weight=1)

win.mainloop() 

Z= 128 2^p-- True
кількість періодів функції f-- 2.0
т. Н.-К. 256.0 >= 32.0 True
v= 4.0 Гц;   T= 0.5 с;   Z= 128
--- 0.008005619049072266 seconds dpf---
--- 0.0 seconds dpf1---
--- 0.0010001659393310547 seconds fft---
--- 0.010998249053955078 seconds ndpf ---
--- 0.0 seconds ifft ---
кількість кроків до основного тону 2 ; точно 2.0
Z= 80 2^p-- False
T0= 0.2 T= 0.16
Z= 64 2^p-- True
кількість періодів функції f-- 0.64
т. Н.-К. 400.0 >= 32.0 True
v= 4.0 Гц;   T= 0.16 с;   Z= 64
--- 0.003067493438720703 seconds dpf---
--- 0.0 seconds dpf1---
--- 0.0 seconds fft---
--- 0.003931522369384766 seconds ndpf ---
--- 0.0 seconds ifft ---
кількість кроків до основного тону 1 ; точно 0.64
Z= 280 2^p-- False
T0= 0.7 T= 0.64
Z= 256 2^p-- True
кількість періодів функції f-- 2.56
т. Н.-К. 400.0 >= 32.0 True
v= 4.0 Гц;   T= 0.64 с;   Z= 256
--- 0.028998613357543945 seconds dpf---
--- 0.0010006427764892578 seconds dpf1---
--- 0.0 seconds fft---
--- 0.04807448387145996 seconds ndpf ---
--- 0.0 seconds iff